In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, LSTM, Dense, Conv1D, MaxPooling1D, Flatten, LeakyReLU, Input, Concatenate
from tensorflow import keras
import tensorflow as tf
import pickle

In [6]:
TRAIN_SET = './rep_train_set.pickle'
TEST_SET = './rep_test_set.pickle'

with open(TEST_SET, 'rb') as file:
    test_set = pickle.load(file)
    x_test = test_set['x']
    y_test = test_set['y']

with open(TRAIN_SET, 'rb') as file:
    train_set = pickle.load(file)
    x_train = train_set['x']
    y_train = train_set['y']
    
x_train = tf.expand_dims(x_train, axis=2)
x_test = tf.expand_dims(x_test, axis=2)
x_train.shape

TensorShape([700432, 187, 1])

In [7]:
y_train_cat = tf.keras.utils.to_categorical(y_train)
y_test_cat = tf.keras.utils.to_categorical(y_test)
print(y_train_cat.shape)

(700432, 8)


In [8]:
print(x_train.shape[1:])

(187, 1)


In [29]:
class SelfSupervisedModel():
    def CNN(self, inputs):
        x = Conv1D(5, kernel_size=3, strides=1)(inputs)
        x = LeakyReLU()(x)
        x = MaxPooling1D(pool_size=2, strides=2)(x)
        x = Conv1D(10, kernel_size=4, strides=1)(x)
        x = LeakyReLU()(x)
        x = MaxPooling1D(pool_size=2, strides=2)(x)
        x = Conv1D(20, kernel_size=4, strides=1)(x)
        x = LeakyReLU()(x)
        x = MaxPooling1D(pool_size=2, strides=2)(x)
        x = Flatten()(x)
        x = Dense(30)(x)
        x = LeakyReLU()(x)
        x = Dense(20)(x)
        x = LeakyReLU()(x)
        x = Dense(1, activation="sigmoid")(x)
        return x
    
    def LSTM(self, inputs):
        x = LSTM(64)(inputs)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.3)(x)
        x = Dense(1, activation='sigmoid')(x)
        return x
    
    def MLP(self, inputs):
        x = Dense(5)(inputs)
        x = Dense(64, activation='relu')(x)
        x = Dense(128, activation='relu')(x)
        x = Dense(128, activation='relu')(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(64, activation='relu')(x)
        x = Flatten()(x)
        x = Dense(1, activation="sigmoid")(x)
        return x

    def assemble(self, input_shape):
        inputs = Input(shape=input_shape)
        cnn = self.CNN(inputs)
        lstm = self.LSTM(inputs)
        mlp = self.MLP(inputs)
        x = Concatenate(axis=1)([cnn, lstm, mlp])
        
        task1 = Dense(1, activation="sigmoid")(x)
        task2 = Dense(1, activation="sigmoid")(x)
        task3 = Dense(1, activation="sigmoid")(x)
        task4 = Dense(1, activation="sigmoid")(x)
        task5 = Dense(1, activation="sigmoid")(x)
        task6 = Dense(1, activation="sigmoid")(x)
        task7 = Dense(1, activation="sigmoid")(x)
        task8 = Dense(1, activation="sigmoid")(x)
        
        model = Model(inputs = inputs, outputs = [task1, task2, task3, task4, task5, task6, task7, task8])
        return model

In [30]:
model = SelfSupervisedModel().assemble(x_train.shape[1:])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 187, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 185, 5)       20          input_6[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_25 (LeakyReLU)      (None, 185, 5)       0           conv1d_15[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_15 (MaxPooling1D) (None, 92, 5)        0           leaky_re_lu_25[0][0]             
____________________________________________________________________________________________

In [31]:
opt = keras.optimizers.SGD(lr=0.003, momentum=0.7)

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [32]:
history = model.fit(x_train, [y_train_cat[:,0],y_train_cat[:,1],y_train_cat[:,2],y_train_cat[:,3],y_train_cat[:,4],y_train_cat[:,5],y_train_cat[:,6],y_train_cat[:,7]],
               batch_size=128,
               epochs=1,
               validation_data=(x_test, [y_test_cat[:,0],y_test_cat[:,1],y_test_cat[:,2],y_test_cat[:,3],y_test_cat[:,4],y_test_cat[:,5],y_test_cat[:,6],y_test_cat[:,7]]),
               shuffle=True)

5473/5473 [==============================] - 1166s 212ms/step - loss: 3.2093 - dense_106_loss: 0.4037 - dense_107_loss: 0.4151 - dense_108_loss: 0.4068 - dense_109_loss: 0.3856 - dense_110_loss: 0.3981 - dense_111_loss: 0.4016 - dense_112_loss: 0.4045 - dense_113_loss: 0.3939 - dense_106_accuracy: 0.8750 - dense_107_accuracy: 0.8749 - dense_108_accuracy: 0.8513 - dense_109_accuracy: 0.8753 - dense_110_accuracy: 0.8749 - dense_111_accuracy: 0.8746 - dense_112_accuracy: 0.8647 - dense_113_accuracy: 0.8750 - val_loss: 2.4026 - val_dense_106_loss: 0.3314 - val_dense_107_loss: 0.3396 - val_dense_108_loss: 0.3235 - val_dense_109_loss: 0.2599 - val_dense_110_loss: 0.2877 - val_dense_111_loss: 0.2134 - val_dense_112_loss: 0.3164 - val_dense_113_loss: 0.3308 - val_dense_106_accuracy: 0.8750 - val_dense_107_accuracy: 0.8750 - val_dense_108_accuracy: 0.8750 - val_dense_109_accuracy: 0.8750 - val_dense_110_accuracy: 0.8750 - val_dense_111_accuracy: 0.8750 - val_dense_112_accuracy: 0.8750 - val_den

In [34]:
model.save("self_model")

INFO:tensorflow:Assets written to: self_model5\assets


INFO:tensorflow:Assets written to: self_model5\assets
